<a href="https://colab.research.google.com/github/paulkroe/open-r1/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Run in Colab
Couldn't get a gpu on gcp

In [5]:
# Clone the repository
!git clone https://github.com/paulkroe/open-r1.git

# Install uv (faster alternative to pip)
!curl -LsSf https://astral.sh/uv/install.sh | sh
!export PATH=$HOME/.local/bin:$PATH  # Ensure uv is available in path

# Install dependencies **inside the project directory**
!cd open-r1 && uv pip install torch==2.5.1 --link-mode=copy --system
!cd open-r1 && uv pip install flash-attn --no-build-isolation --no-cache-dir --system
!cd open-r1 && uv pip install --upgrade pip --link-mode=copy --system
!cd open-r1 && uv pip install vllm==0.7.1 --link-mode=copy --system

# Now install project dependencies from `setup.py` in open-r1
!cd open-r1 && GIT_LFS_SKIP_SMUDGE=1 uv pip install -e ".[dev]" --link-mode=copy --system


fatal: destination path 'open-r1' already exists and is not an empty directory.
downloading uv 0.5.31 x86_64-unknown-linux-gnu
no checksums to verify
installing to /root/.local/bin
  uv
  uvx
everything's installed!
Using Python 3.11.11 environment at: /usr
Audited 1 package in 76ms
Using Python 3.11.11 environment at: /usr
Audited 1 package in 78ms
Using Python 3.11.11 environment at: /usr
Resolved 1 package in 41ms
Audited 1 package in 0.09ms
Using Python 3.11.11 environment at: /usr
Resolved 132 packages in 63ms
Uninstalled 2 packages in 17ms
Installed 2 packages in 692ms
 - compressed-tensors==0.9.1
 + compressed-tensors==0.9.0
 - vllm==0.7.2
 + vllm==0.7.1
Using Python 3.11.11 environment at: /usr
   Updating https://github.com/huggingface/trl.git (main)
   Updating https://github.com/huggingface/trl.git (main)
   Updating https://github.com/huggingface/trl.git (main)
   Updating https://github.com/huggingface/transformers.git (main)
   Updating https://github.com/huggingface/trl.

In [2]:
# Authenticate Hugging Face and Weights & Biases
!huggingface-cli login
!wandb login

# Install Git LFS (for handling large model files)
!sudo apt-get install git-lfs -y
!git-lfs --version  # Verify installation


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `agentic-ciengine` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `agentic-

In [3]:
!uv pip install vllm transformers accelerate --upgrade --system
!uv pip install "distilabel[vllm]>=1.5.2" datasets --system

Using Python 3.11.11 environment at: /usr
Resolved 133 packages in 361ms
⠙ Preparing packages... (0/14)
⠙ Preparing packages... (0/14)
anyio      ------------------------------     0 B/93.79 KiB
⠙ Preparing packages... (0/14)
anyio      ------------------------------     0 B/93.79 KiB
google-api-core ------------------------------     0 B/156.31 KiB
⠙ Preparing packages... (0/14)
aiohappyeyeballs ------------------------------     0 B/14.20 KiB
anyio      ------------------------------     0 B/93.79 KiB
google-api-core ------------------------------     0 B/156.31 KiB
⠙ Preparing packages... (0/14)
aiohappyeyeballs ------------------------------     0 B/14.20 KiB
anyio      ------------------------------ 14.88 KiB/93.79 KiB
google-api-core ------------------------------     0 B/156.31 KiB
⠙ Preparing packages... (0/14)
aiohappyeyeballs ------------------------------     0 B/14.20 KiB
anyio      ------------------------------ 14.88 KiB/93.79 KiB
google-api-core -------------------------

In [6]:
!tail -f /root/.cache/distilabel/pipelines/distill-qwen-1.5b-r1/*/executions/*/log.txt

tail: cannot open '/root/.cache/distilabel/pipelines/distill-qwen-1.5b-r1/*/executions/*/log.txt' for reading: No such file or directory
tail: no files remaining


In [1]:
from datasets import load_dataset
from distilabel.models import vLLM
from distilabel.pipeline import Pipeline
from distilabel.steps.tasks import TextGeneration

# ✅ Use a smaller model for T4 compatibility
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# 📝 Define prompt format
prompt_template = """\
You will be given a problem. Please reason step by step, and put your final answer within \boxed{}:
{{ instruction }}"""

# 📥 Load dataset (limit to 10 examples for testing)
dataset = load_dataset("AI-MO/NuminaMath-TIR", split="train").select(range(10))

# 🏗️ Define the pipeline
with Pipeline(
    name="distill-qwen-1.5b-r1",
    description="Generating data from DeepSeek-R1-Distill-Qwen-1.5B",
) as pipeline:

    llm = vLLM(
        model=model_id,
        tokenizer=model_id,
        dtype="bfloat16",  # ✅ Correct placement
        extra_kwargs={
            "tensor_parallel_size": 1,  # ✅ Keep other options
            "max_model_len": 4096,
        },
        generation_kwargs={
            "temperature": 0.6,
            "max_new_tokens": 512,
        },
    )

    # 🔁 Define text generation step
    prompt_column = "problem"
    text_generation = TextGeneration(
        llm=llm,
        template=prompt_template,
        num_generations=2,  # Reduce memory usage
        input_mappings={"instruction": prompt_column} if prompt_column else {}
    )

distiset = pipeline.run(dataset=dataset, use_cache=False)
print(distiset)
distiset.save_to_disk("generated_data")


ModuleNotFoundError: No module named 'datasets'

In [23]:
!rm -rf generated_data

In [37]:
import os
from datasets import load_dataset

dataset = load_dataset("generated_data", "default")
print(dataset["train"][0])


{'_data_files': [{'filename': 'data-00000-of-00001.arrow'}], '_fingerprint': '8f0d1677e2deb4f2', '_format_columns': None, '_format_kwargs': {}, '_format_type': None, '_output_all_columns': False, '_split': 'train'}
train
